In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import os
input_fname = 'test_IOTA.in'
np.set_printoptions(precision=3)

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [3]:
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=10
beam.current=0.0
dist = beam.distribution
dist.distribution_type = 'Waterbag'
lattice = [impact.getElem('loop_through_lattice'),
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl')]
lattice[0].nturn = 1
lattice[1].file_id = 1001
lattice[3].file_id = 1002
lattice[5].file_id = 1003


ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

In [4]:
dist.emitx = dist.emitx*10000
dist.emity = dist.emity*10000

# 1. Drift

In [5]:
elemF = impact.getElem('drift')
elemF.length = 1.1
elemF.n_sckick = 20
elemF.n_map = 2

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

input error <- sum(beam.multi_charge.n_particles) not qual to beam.n_particles
  ... enforcing  beam.multi_charge.n_particles[0] to beam.n_particles
input error <- beam.multi_charge.current[0] not qual to beam.current
  ... enforcing  beam.multi_charge.current[0] to beam.current


0

In [6]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00  0.000e+00  1.735e-17  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.735e-17  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]]


# 2. Quad

In [7]:
elemF = impact.getElem('quad')
elemF.length = 0.02
elemF.n_sckick = 20
elemF.n_map = 1
elemF.B1 = 15.0

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [8]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-5.551e-16  2.776e-16 -7.980e-17  2.220e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  9.714e-16 -8.049e-16  0.000e+00  0.000e+00]
 [-1.388e-16  0.000e+00  4.441e-16 -3.886e-16  0.000e+00  0.000e+00]
 [ 1.665e-16  0.000e+00 -4.163e-16  8.049e-16  0.000e+00  0.000e+00]
 [ 4.163e-17 -5.551e-17 -6.245e-17  5.551e-16  0.000e+00  0.000e+00]
 [ 0.000e+00 -1.943e-16  3.253e-18 -2.498e-16  0.000e+00  0.000e+00]
 [-4.163e-17 -1.110e-16  8.049e-16 -4.163e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  5.551e-17  3.053e-16 -8.327e-17  0.000e+00  0.000e+00]
 [ 0.000e+00 -4.163e-16 -4.718e-16  2.776e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  1.110e-16 -3.053e-16  3.053e-16  0.000e+00  0.000e+00]]


# 3. Sextupole

In [9]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'multipole_thin':
        break
lattice0[i]


 KL_dipole: 0.0
   KL_quad: 0.0
   KL_sext: 0.15
    KL_oct: 0.0
   KL_deca: 0.0
 KL_dodeca: 0.0
      type: 'multipole_thin'

In [10]:
elemF = lattice0[i]

elemB = copy(elemF)
elemF.KL_sext = 1000.0
elemB.KL_sext =-elemF.KL_sext

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [11]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00        nan  0.000e+00 -4.996e-16        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00  4.441e-16        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00  2.331e-15        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -1.665e-15        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00  8.327e-17        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -2.776e-17        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -4.163e-16        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -3.469e-17        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -2.040e-15        nan  0.000e+00]
 [ 0.000e+00        nan  0.000e+00 -1.332e-15        nan  0.000e+00]]


# 4. Dipole

In [12]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'dipole':
        break
lattice0[i]


                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'

### 4.1 without fringe field

##### negative length 

In [13]:
elemF = copy(lattice0[i])
elemF.fringe_field_integration = 0.0

elemB = copy(elemF)
elemB.length = -elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [14]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 2.891e-13 -7.167e-13  0.000e+00  0.000e+00  2.397e+02  0.000e+00]
 [-1.096e-12  2.713e-12  0.000e+00  0.000e+00 -1.465e+02  0.000e+00]
 [-1.396e-12  3.455e-12  0.000e+00  0.000e+00  1.546e+02  0.000e+00]
 [ 1.058e-12 -2.619e-12  0.000e+00  0.000e+00 -9.348e+01  0.000e+00]
 [ 1.994e-12 -4.933e-12  0.000e+00  0.000e+00 -1.084e+02  0.000e+00]
 [ 3.149e-13 -7.795e-13  6.288e-18  0.000e+00  1.467e+02  0.000e+00]
 [ 9.007e-13 -2.229e-12  0.000e+00  0.000e+00  1.056e+02  0.000e+00]
 [-1.160e-12  2.871e-12  0.000e+00  0.000e+00  6.892e+01  0.000e+00]
 [ 1.907e-12 -4.718e-12  0.000e+00  0.000e+00 -1.816e+02  0.000e+00]
 [-9.623e-13  2.381e-12  0.000e+00  0.000e+00 -7.756e+01  0.000e+00]]


### negative length and bending angle

In [15]:
elemB.bending_angle = -elemF.bending_angle
impact.writeInputFile(beam,lattice);
impact.run()

0

In [16]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-2.220e-16  0.000e+00  0.000e+00  0.000e+00 -1.066e-13  0.000e+00]
 [ 3.331e-16 -4.163e-16  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.665e-16  1.527e-16  0.000e+00  0.000e+00 -8.527e-14  0.000e+00]
 [ 1.665e-16 -1.388e-16  0.000e+00  0.000e+00  4.263e-14  0.000e+00]
 [-1.110e-16 -5.551e-17  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-3.331e-16  2.776e-17  6.288e-18  0.000e+00  0.000e+00  0.000e+00]
 [-1.041e-16 -2.498e-16  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-6.592e-17 -5.551e-17  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 2.776e-16 -1.110e-16  0.000e+00  0.000e+00  5.684e-14  0.000e+00]
 [ 3.053e-16  1.110e-16  0.000e+00  0.000e+00  1.137e-13  0.000e+00]]


### 4.2 with fringe field

In [17]:
elemF = copy(lattice0[i])

elemB = copy(elemF)
elemB.length = -elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [18]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 2.891e-13 -7.167e-13  3.494e-03 -1.728e-03  2.397e+02  0.000e+00]
 [-1.096e-12  2.713e-12  7.055e-03 -3.712e-02 -1.465e+02  0.000e+00]
 [-1.396e-12  3.455e-12  6.605e-03 -2.950e-02  1.546e+02  0.000e+00]
 [ 1.058e-12 -2.619e-12 -4.038e-05  1.792e-02 -9.348e+01  0.000e+00]
 [ 1.994e-12 -4.933e-12  6.313e-03 -5.145e-03 -1.084e+02  0.000e+00]
 [ 3.149e-13 -7.795e-13 -8.114e-03  1.037e-02  1.467e+02  0.000e+00]
 [ 9.007e-13 -2.229e-12  7.743e-03 -3.469e-02  1.056e+02  0.000e+00]
 [-1.160e-12  2.871e-12  7.252e-03 -2.166e-02  6.892e+01  0.000e+00]
 [ 1.907e-12 -4.718e-12 -8.541e-03  3.163e-02 -1.816e+02  0.000e+00]
 [-9.623e-13  2.381e-12 -3.811e-03  2.033e-02 -7.756e+01  0.000e+00]]


### negative length, bending angle and FINT

###### MaryLie_order = 1

In [19]:
elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
impact.writeInputFile(beam,lattice);
impact.run()
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-2.220e-16  0.000e+00 -7.321e-16  1.509e-14 -1.066e-13  0.000e+00]
 [ 3.331e-16 -4.163e-16 -7.133e-15  1.509e-13  0.000e+00  0.000e+00]
 [ 1.665e-16  1.527e-16 -4.802e-15  1.012e-13 -8.527e-14  0.000e+00]
 [ 1.665e-16 -1.388e-16 -1.277e-15  2.573e-14  4.263e-14  0.000e+00]
 [-1.110e-16 -5.551e-17  6.661e-16 -1.407e-14  0.000e+00  0.000e+00]
 [-3.331e-16  2.776e-17  2.385e-17 -4.996e-16  0.000e+00  0.000e+00]
 [-1.041e-16 -2.498e-16  6.051e-15 -1.273e-13  0.000e+00  0.000e+00]
 [-6.592e-17 -5.551e-17  2.692e-15 -5.854e-14  0.000e+00  0.000e+00]
 [ 2.776e-16 -1.110e-16 -7.300e-15  1.568e-13  5.684e-14  0.000e+00]
 [ 3.053e-16  1.110e-16 -5.829e-16  1.082e-14  1.137e-13  0.000e+00]]


###### MaryLie_order = 2

In [20]:
elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
os.system('xmain_dipoleF2')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-2.220e-16  0.000e+00 -7.321e-16  1.509e-14 -1.066e-13  0.000e+00]
 [ 3.331e-16 -4.163e-16 -7.133e-15  1.509e-13  0.000e+00  0.000e+00]
 [ 1.665e-16  1.527e-16 -4.802e-15  1.012e-13 -8.527e-14  0.000e+00]
 [ 1.665e-16 -1.388e-16 -1.277e-15  2.573e-14  4.263e-14  0.000e+00]
 [-1.110e-16 -5.551e-17  6.661e-16 -1.407e-14  0.000e+00  0.000e+00]
 [-3.331e-16  2.776e-17  2.385e-17 -4.996e-16  0.000e+00  0.000e+00]
 [-1.041e-16 -2.498e-16  6.051e-15 -1.273e-13  0.000e+00  0.000e+00]
 [-6.592e-17 -5.551e-17  2.692e-15 -5.854e-14  0.000e+00  0.000e+00]
 [ 2.776e-16 -1.110e-16 -7.300e-15  1.568e-13  5.684e-14  0.000e+00]
 [ 3.053e-16  1.110e-16 -5.829e-16  1.082e-14  1.137e-13  0.000e+00]]


###### MaryLie_order = 3

In [21]:
elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
impact.writeInputFile(beam,lattice);
os.system('xmain_dipoleF3')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-2.220e-16  0.000e+00 -7.321e-16  1.509e-14 -1.066e-13  0.000e+00]
 [ 3.331e-16 -4.163e-16 -7.133e-15  1.509e-13  0.000e+00  0.000e+00]
 [ 1.665e-16  1.527e-16 -4.802e-15  1.012e-13 -8.527e-14  0.000e+00]
 [ 1.665e-16 -1.388e-16 -1.277e-15  2.573e-14  4.263e-14  0.000e+00]
 [-1.110e-16 -5.551e-17  6.661e-16 -1.407e-14  0.000e+00  0.000e+00]
 [-3.331e-16  2.776e-17  2.385e-17 -4.996e-16  0.000e+00  0.000e+00]
 [-1.041e-16 -2.498e-16  6.051e-15 -1.273e-13  0.000e+00  0.000e+00]
 [-6.592e-17 -5.551e-17  2.692e-15 -5.854e-14  0.000e+00  0.000e+00]
 [ 2.776e-16 -1.110e-16 -7.300e-15  1.568e-13  5.684e-14  0.000e+00]
 [ 3.053e-16  1.110e-16 -5.829e-16  1.082e-14  1.137e-13  0.000e+00]]


# 5. Nonlinear Insert

In [22]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'nonlinear_insert':
        break
lattice0[i]


             length: 1.8 [m]
           n_sckick: 96 [1]
              n_map: 5 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
       tune_advance: 0.3034496449165134
        pipe_radius: 10.0 [m]
               type: 'nonlinear_insert'

In [23]:
elemF = lattice0[i]

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [24]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00 -1.665e-16  1.735e-17  0.000e+00  1.421e-14  0.000e+00]
 [ 4.996e-16 -5.551e-16  6.661e-16 -1.388e-16  0.000e+00  0.000e+00]
 [-3.053e-16  0.000e+00  1.388e-16  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -8.327e-17 -4.441e-16  0.000e+00  0.000e+00]
 [ 2.498e-16  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  1.670e-16  0.000e+00  0.000e+00  0.000e+00]
 [-2.845e-16  0.000e+00  1.388e-16  0.000e+00 -5.684e-14  0.000e+00]
 [ 3.018e-16  0.000e+00  1.527e-16  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  4.996e-16  0.000e+00  0.000e+00]
 [ 1.388e-16  0.000e+00  1.943e-16  1.388e-16  0.000e+00  0.000e+00]]


# 6. Linear matrix map (toy model compoenent)

In [25]:
elemF = impact.getElem('linear_matrix_map')
elemF.nonlinear_insert_length = 1.8
elemF.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF.tune_advance = 0.22

elemB = copy(elemF)
elemB.tune_advance =-elemF.tune_advance
elemB.nonlinear_insert_tuneAdvance =-elemF.nonlinear_insert_tuneAdvance
elemB.nonlinear_insert_length =-elemF.nonlinear_insert_length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [26]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00 -1.665e-16  0.000e+00 -5.551e-17  0.000e+00  0.000e+00]
 [-1.110e-16  1.388e-16 -1.110e-16  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  1.388e-17  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  1.388e-16  1.388e-16  0.000e+00  0.000e+00  0.000e+00]
 [ 5.551e-17 -5.551e-17  4.857e-17 -5.551e-17  0.000e+00  0.000e+00]
 [ 1.665e-16 -5.551e-17 -4.922e-17  2.776e-17  0.000e+00  0.000e+00]
 [ 5.551e-17  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  5.551e-17 -1.388e-17  6.939e-18  0.000e+00  0.000e+00]
 [-1.665e-16  0.000e+00  3.331e-16  0.000e+00  0.000e+00  0.000e+00]
 [-1.943e-16  1.110e-16  8.327e-17 -5.551e-17  0.000e+00  0.000e+00]]


# 7. ToyModel

In [27]:
elemF = impact.getElem('nonlinear_insert')
elemF.tune_advance = 0.3034496449165134
elemF.length = 1.8
elemF.n_sckick = 100
elemF.n_map = 10
elemF.strength_t = 0.4
elemF.transverse_scale_c = 0.01

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

In [28]:
elemF1 = impact.getElem('linear_matrix_map')
elemF1.nonlinear_insert_length = 1.8
elemF1.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF1.tune_advance = 0.0

elemB1 = copy(elemF1)
elemB1.tune_advance =-elemB1.tune_advance
elemB1.nonlinear_insert_tuneAdvance =-elemB1.nonlinear_insert_tuneAdvance
elemB1.nonlinear_insert_length =-elemB1.nonlinear_insert_length

lattice.insert(3,elemF1)
lattice.insert(5,elemB1)

In [29]:
for i in range(len(lattice)):
    print(lattice[i].type)

loop_through_lattice
write_raw_ptcl
nonlinear_insert
linear_matrix_map
write_raw_ptcl
linear_matrix_map
nonlinear_insert
write_raw_ptcl


In [30]:
impact.writeInputFile(beam,lattice);
impact.run()

0

In [31]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-7.216e-16  0.000e+00  0.000e+00 -2.776e-17  0.000e+00  0.000e+00]
 [ 6.661e-16 -4.163e-16  0.000e+00 -2.776e-16  0.000e+00  0.000e+00]
 [ 1.665e-16 -2.776e-17  1.388e-16  0.000e+00  0.000e+00  0.000e+00]
 [-1.388e-16  0.000e+00 -5.274e-16  5.551e-16  0.000e+00  0.000e+00]
 [ 3.608e-16 -2.776e-17 -3.469e-17  0.000e+00  0.000e+00  0.000e+00]
 [ 3.053e-16  0.000e+00  3.246e-16  0.000e+00  0.000e+00  0.000e+00]
 [-3.608e-16  0.000e+00  1.388e-16  0.000e+00  0.000e+00  0.000e+00]
 [ 1.596e-16 -2.776e-17  1.527e-16  0.000e+00  0.000e+00  0.000e+00]
 [-4.996e-16  0.000e+00 -3.053e-16  2.637e-16  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -7.494e-16  4.441e-16  0.000e+00  0.000e+00]]
